#THREADS

In [ ]:
import threading

def print_message():
    print("I am a Thread!")

thread = threading.Thread(target=print_message)

#Para iniciar el thread
thread.start()

#Para esperar a que el thread termine
#thread.join()

I am a Thread!


#El programa termina; pero por el sleep hace que el thread no se haya ejecutado antes de que termine la funcion, y se ejecuta despues del sleep

In [5]:
import threading
import time

def print_message():
    time.sleep(10)
    print("I am a Thread!")

thread = threading.Thread(target=print_message)

#Para iniciar el thread
thread.start()
print("Thread finished!")

Thread finished!


I am a Thread!


El join va a bloquear la ejecucion hasta que no haya terminado la ejecucion del thread; como una barrera, no puede continuar con lo que viene despies hasta que el thread no haya terminado

In [6]:
import threading
import time

def print_message():
    time.sleep(10)
    print("I am a Thread!")

thread = threading.Thread(target=print_message)

#Para iniciar el thread
thread.start()

thread.join()
print("Thread finished!")

I am a Thread!
Thread finished!


Si le queremos pasar parametros:

In [ ]:
import threading
import time

def print_message(num):
    #time.sleep(10)
    print(f"I am a Thread {num}")

#le pasamos el numero como tupla
thread = threading.Thread(target=print_message, args=(1,))

#Para iniciar el thread
thread.start()

thread.join()
print("Thread finished!")

I am a Thread! 1
Thread finished!


Con mas threads

In [ ]:
import threading
import time

def print_message(num):
    print(f"I am a Thread {num}")
    #time.sleep(5)

threads = []

#le pasamos el numero como tupla
for i in range(5):
    thread = threading.Thread(target=print_message, args=(i,))
    thread.start()
    threads.append(thread)
    
#En vez de ir esperando a que cada thread termine de ejecutarse para ejecutar el siguiente; creo una lista de threads y 
# cuando ya se han ejecutado, entonces imprimo "Thread finished!"
for thread in threads:
    thread.join()

print("Thread finished!")

I am a Thread 0
I am a Thread 1
I am a Thread 2
I am a Thread 3
I am a Thread 4
Thread finished!


Prueba; hay veces que sale mal, porque se realiza el cambio de contexto mientras uno de los threads se esta imprimiendo su mensaje

In [ ]:
import threading
import time

def print_message(num):
    if (num == 3):
        time.sleep(5)
    print(f"I am a Thread {num}")

threads = []

#le pasamos el numero como tupla
for i in range(5):
    thread = threading.Thread(target=print_message, args=(i,))
    thread.start()
    threads.append(thread)
    
#En vez de ir esperando a que cada thread termine de ejecutarse para ejecutar el siguiente; creo una lista de threads y 
# cuando ya se han ejecutado, entonces imprimo "Thread finished!"
for thread in threads:
    thread.join()

print("Thread finished!")

I am a Thread 0
I am a Thread 1
I am a Thread 2
I am a Thread 4
I am a Thread 3
Thread finished!


Semaforos

In [1]:
import threading
import time

#Creamos el semaforo; el 2 es para indicar el numero maximo de hilos que pueden acceder a la seccion critica a la vez
sem = threading.Semaphore(1)

filename = "log2.txt"

def access_resource(num):
    #Adquiere el semaforo, restamos uno a ese semaforo
    sem.acquire()
    for i in range(5):
        with open(filename, "a") as file:
            file.write(f"Thread {num} -> iter {i}\n")
    print("\n")
    #Este semaforo ha terminado de ejecutar la seccion critica, entonces sumamos uno para que otro hilo pueda usarlo
    sem.release()

threads = []

#le pasamos el numero como tupla
for i in range(5):
    thread = threading.Thread(target=access_resource, args=(i,))
    thread.start()
    threads.append(thread)
    
#En vez de ir esperando a que cada thread termine de ejecutarse para ejecutar el siguiente; creo una lista de threads y 
# cuando ya se han ejecutado, entonces imprimo "Thread finished!"
for thread in threads:
    thread.join()

Seria lo mismo con LOCK

In [3]:
import threading
import time

#Creamos el semaforo; el 2 es para indicar el numero maximo de hilos que pueden acceder a la seccion critica a la vez
lock = threading.Lock()

filename = "log2.txt"

def access_resource(num):
    #Adquiere el semaforo, restamos uno a ese semaforo
    lock.acquire()
    for i in range(5):
        with open(filename, "a") as file:
            file.write(f"Thread {num} -> iter {i}\n")
    print("\n")
    #Este semaforo ha terminado de ejecutar la seccion critica, entonces sumamos uno para que otro hilo pueda usarlo
    lock.release()

threads = []

#le pasamos el numero como tupla
for i in range(5):
    thread = threading.Thread(target=access_resource, args=(i,))
    thread.start()
    threads.append(thread)
    
#En vez de ir esperando a que cada thread termine de ejecutarse para ejecutar el siguiente; creo una lista de threads y 
# cuando ya se han ejecutado, entonces imprimo "Thread finished!"
for thread in threads:
    thread.join()

El interprete de python no permite que haya mas de un nucleo de ejecucion para los hilos ¿?

parece que se esta ejecutando secuencialmente; los ejercicios anteriores se ejecutan en el mismo nucleo

GLI (Global Interpreter Lock)

Proceso ==> No comparten la memoria

Hilo ==> los hilos se ejecutan dentro del mismo proceso, si creo 10 hilos comparten la misma zona de memoria

In [ ]:
import threading
import time

def countdown(n):
    while n > 0:
        n -= 1

count = 500000000
countdown(count)

El hacer que ese count lo haga dentro de un thread, tarda un poco mas por la creacion de un thread. Lo que pasa es q cuando tenemos dos threads, en este caso, no se dividiria el trabajo, sino que seria mas lento

In [9]:
thread1 = threading.Thread(target=countdown, args=(count,))
thread2 = threading.Thread(target=countdown, args=(count,))
thread1.start()
thread2.start()
thread1.join()
thread2.join()

Crear threads no me sirve para optimizar en Python; creamos threads para que los procesos que son bloqueantes, dejen de ser bloqueantes

Como se realiza una barra de progreso